# Potential Player
## Objective
Identify users not playing any Nexon game, but have high likelihood of playing one of Nexon's game

## Methodology
Nexon has 3 games, each with its own player profile. We will try to identify players sharing similar profile, such as purchasing and gaming behavior.  
The assumption is that if, for example, most MapleStory players only spend about 30 hours playing a week, it could mean that the nature of the game or the gaming community is suitable for people who can devote 30 hours a week playing game. Thus, people who do not spend around the same time are likely not suitable for MapleStory and therefore have less likelihood of playing MapleStory.  
Here we will create user profile (features) and build binary classification model to see the likelihood of players for playing each Nexon game (MapltStory, Mabinogi, and Vindictus). Our target here are players who have high probability but had not play at Nexon game before.

## Steps
1. Label games with relevant genre to slim down number of available features
2. feature genaration such as purchase genre Y, play time for genre X, % time playing genre X, # of games played, ...etc.
3. model building

## Brainstorm/Notes
1. look into people who play low-res games (games with earlier published days) as they may have hardware limtations
2. look into people who play exclusively MMORPG

## Obsticles
1. There's no timestamp for time information. 
    * Unable to distinguish between players who play lots of hours in short period of time vs one who plays moderate amount but over a long horizon.
    * Unable to tell if certain game was played recently and if a player has moved onto to a different genre of games

In [1]:
import numpy as np
import pandas as pd
import utility

In [2]:
# read data
df = pd.read_csv('data/steam-200k.csv', header=None)\
       .rename(columns={0:'id', 1:'game', 2:'action', 3:'value'})\
       .astype({'id':str})\
       .iloc[:, :4] # drop last column
df = df.drop_duplicates()
df.head(3)

,id,game,action,value
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0


## EDA
We will perform analysis on players who played Nexon game (MapleStory, Mabinogi, Vindictus) vs those who have nevery played Nexon game  
  
__Note:__ when we flag Nexon player, we only flags players who have "play" action. Users with only "purchase" action and no play action is not considered Nexon player.

In [4]:
# flag players who has played Nexon game
df = utility.flag_nexon_player(df)

In [14]:
# here we run the same general EDA on Nexon player
# actual EDA code are in utility.py, calling function here for readability
df_nexon = df[df['flg_NX'] == 1].reset_index(drop=True)
utility.eda_general(df_nexon)

Number of records:          5824
Number of unique User ID:   45
Number of games:            1560

	Action Count: 
purchase    3801
play        2023
Name: action, dtype: int64

Number of Purchasing Player: 45
% of Purchasing Player: 100.0%

	Summary Statistics on Play Time:
count    2023.000000
mean       23.405685
std        91.997070
min         0.100000
25%         0.900000
50%         3.800000
75%        13.300000
max      2563.000000
Name: value, dtype: float64

	Summary Statistics on Number of Games Played:
count    2023.000000
mean       82.405833
std        49.377679
min         1.000000
25%        38.000000
50%        83.000000
75%       109.000000
max       169.000000
Name: game_freq, dtype: float64

	Summary Statistics on Number of Games Purchase:
count    3801.000000
mean      240.102341
std       222.800806
min         3.000000
25%        78.000000
50%       139.000000
75%       413.000000
max       666.000000
Name: purch_freq, dtype: float64

	Top 10 Games Purchased
Team F

In [15]:
# general EDA on non-Nexon player
df_non_NX = df[df['flg_NX'] != 1]
utility.eda_general(df_non_NX)

Number of records:          193469
Number of unique User ID:   12348
Number of games:            5116

	Action Count: 
purchase    125003
play         68466
Name: action, dtype: int64

Number of Purchasing Player: 11305
% of Purchasing Player: 92.0%

	Summary Statistics on Play Time:
count    68466.000000
mean        49.630709
std        232.118551
min          0.100000
25%          1.000000
50%          4.500000
75%         19.300000
max      11754.000000
Name: value, dtype: float64

	Summary Statistics on Number of Games Played:
count    68454.000000
mean        56.220761
std         74.122473
min          1.000000
25%          8.000000
50%         29.000000
75%         76.000000
max        498.000000
Name: game_freq, dtype: float64

	Summary Statistics on Number of Games Purchase:
count    125003.000000
mean        129.299897
std         179.676564
min           1.000000
25%          15.000000
50%          59.000000
75%         163.000000
max        1068.000000
Name: purch_freq, dty

### EDA Summary
Some rudimentary analysis comparing summary statitics between Nexon players and non-Nexon players shows that Nexon players on average have played more games, purchased more games, but spend less time playing games.  
The game purchased profile is also quite different with Nexon players having more presence of Sandbox type of games. Although one can certainly argue that those games are popular games , including Nexon players, play them.